<a href="https://colab.research.google.com/github/avilla03/Convolutional-NN-Project/blob/main/train%26test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tqdm.auto import tqdm
torch.manual_seed(12)
torch.cuda.manual_seed(12)
epochs = 50
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
for epoch in tqdm(range(epochs)):
  print(f" Model has seen data {epoch} times \n -------------")
  train_loss = 0
  train_acc = 0
  test_loss = 0
  test_acc = 0
  for batch, (X,y) in enumerate(train_subset_dataloader):
    X,y = X.to(device),y.to(device)
    y = y.to(torch.float32)
    model.train()
    y_logits = model.forward(X).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits))
    loss = loss_fn(y_logits,y)
    acc = accuracy_fn(y_true = y, y_pred = y_pred)
    train_loss += loss
    train_acc += acc
    for param in model.parameters():
      param.grad = None
    loss.backward()
    optimizer.step()
    #lr_scheduler.step(loss)
    if batch % 200 == 0:
      print(f"Looked at {batch * len(X)} / {len(train_subset_dataloader.dataset)} samples.")
  train_loss /= len(train_subset_dataloader) # average per batch
  train_acc /= len(train_subset_dataloader)
  test_loss, test_acc = 0,0
  model.eval()
  with torch.inference_mode():
    for X_test,y_test in test_subset_dataloader:
      X_test, y_test = X_test.to(device), y_test.to(device)
      y_test = y_test.to(torch.float32).squeeze()
      test_logits = model(X_test).squeeze()
      test_pred = torch.round(torch.sigmoid(test_logits))
      test_loss += loss_fn(test_logits,y_test)
      test_acc += accuracy_fn(y_true = y_test, y_pred = test_pred)
    test_loss /= len(test_subset_dataloader)
    test_acc /= len(test_subset_dataloader)
    #lr_scheduler.step(test_loss)
    print(f"\nTrain loss avg: {train_loss: .4f} | Train acc avg: {train_acc: .4f} | Test loss avg: {test_loss: .4f}, Test acc avg: {test_acc: .4f}")